## Now I got the data for the results with Tensorflow dence layers. I categorized good MAPE result country and not good one. 

Now I will train category 2 with linear regression to get better MAPE results. 

### Things to do:

* Load result data with category. 
* Load train data. 
* Grab the cfips on category 2.
* train with linear regression if the country incategory 2, then update MAPE. 


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [16]:
data = pd.read_csv("result_dense_with_category")
train = pd.read_csv("train.csv")

In [17]:
# Grab the cfips on category 2
cfips_category2 = data[data['category'] == 2]['Country'].unique()

In [13]:
THRESHOLD = 0.103

IDS = train.cfips.unique()
x_train = np.arange(39).reshape((-1,1))
x_test = np.arange(38,47).reshape((-1,1))

# Based on the Chris's rule, SMAPE > THRESHOLD -> Use linear regrssion 
lr_or_non_lr = []  # If use linear regression, label as 1, if not label as 0
smape = []
mape = []
predict = []
for cfips in IDS:
    c = cfips
    df = train.loc[train['cfips'] == c]
    last = df['microbusiness_density'].values[-1]

    # Fit linear regression
    model = LinearRegression()
    model.fit(x_train,df['microbusiness_density'])
    p = model.predict(x_train)

    # Compute train error
    err = p - df['microbusiness_density'].values
    rng = df['microbusiness_density'].max() - df['microbusiness_density'].min()
    #print(f'Error: {err}\nRange: {rng}')

    # Determin if time series is linear or not
    s = 0
    for k in range(39):
        e = np.abs(err[k])
        #print(f'e : {e}')
        r = e / rng # absolute error divided by range
        #print(f'r: {r}')
        s += r
    mape.append(s) # Grab MAPE to compare with other data
    #print(f's: {s}')
    s = s/39 # now S is MAPE mean absolute percentage error
    smape.append(s) 
    #print(f'Divide s by the number of data -> MAPE: {s}')

    # Infer test data with linear regression
    p2 = model.predict(x_test)
    shift = last - p2[0]
    if s <THRESHOLD: 
        preds = p2[1:]+shift
        label = 0
        lr_or_non_lr.append(label)
        smape.append(s)
        predict.append(preds)
    else: 
        preds = [last]*8
        label = 1
        lr_or_non_lr.append(label)
        smape.append(s)
        predict.append(preds)

count_lr = lr_or_non_lr.count(0)
count_non_lr = lr_or_non_lr.count(1)
print(f'The data predict with Linear Regression: {count_lr}')
print(f'The data not predict with Linear Regression: {count_non_lr}')
print(f'Model: {model}\nMean SMAPE: {np.mean(smape)}\nMin SMAPE: {np.min(smape)}\nMax SMAPE: {np.max(smape)}\nMean MAPE: {np.mean(mape)}')

The data predict with Linear Regression: 491
The data not predict with Linear Regression: 2644
Model: LinearRegression()
Mean SMAPE: 0.15733848411716167
Min SMAPE: 0.025774969877374484
Max SMAPE: 0.34610839978243374
Mean MAPE: 6.136200880569304


### Work on this for loop to get new MAPE with new training. Let's see if the total MAPE increase. 

Code above is the Renear regression training. 

In [22]:
mape = []

for i in range(len(data)):
    c = data['Country'].iloc[i]
    # If the cfips in the category 2, then train with linear regression and update MAPE
    if c in cfips_category2:
        
        mape.append(data["MAPE"].iloc[i])
    # If category 1, keep the MAPE score
    else:
        mape.append(data['MAPE'].iloc[i])
        
data['new_MAPE'] = mape
data[['MAPE', 'new_MAPE']]

,MAPE,new_MAPE
0,1.504614,1.504614
1,4.143341,4.143341
2,1.395184,1.395184
3,5.228748,5.228748
4,1.442244,1.442244
...,...,...
3130,8.937141,8.937141
3131,3.437833,3.437833
3132,4.339747,4.339747
3133,2.373714,2.373714
